<a id='top'></a>

# NLP 2 - Compare Reviewers Writing Styles via Cosine Angles of Vectors

#### This project uses Natural Language Tool-Kit (NLTK) for text preprocessing, then assesses the similarity of document pairs using the cosine of the angle between two vectors.

Representing documents in vector space based on the counts of words in the documents (see NLP 1) may mistakenly classify documents with similar content as different, due to difference in document size. Computing the cosine similarity of the vector representations of two documents compensates for the effect of document length when quantifying their similarity. This natural language processing project compares the cosine of the angles between sets of three documents (in three pairs) to see if writing styles are similar or different. The goal was to see if expected differences and similarities in writing styles were borne out by computing and comparing the cosine similarity of the vector representations of pairs of documents. 

The documents being compared are movie reviews downloaded from rottentomatoes.com. After some data wrangling, each document is the entirety of a particular reviewers' text reviews. 

For a full description of what documents are being compared and why, see the blog post for this project.

- [Step 1](#Step_1) Data wrangling. Select text of reviewers that are being compared and combine all their reviews into a dataframe with different rows for each reviewer, and a column that is the entire text of their reviews joined into a single string. For each reviewer, create a variable representing the string of all their reviews joined together.  
- [Step 2](#Step_2) Process the text for analysis. Create a function to process text for analysis, including tokenizing (break words into components), stemming with nltk PorterStemmer (consolidate different permutations of words), and removing stopwords (specific articles, prepositions and other words that do not add meaning).
- [Step 3](#Step_3) Compute cosine similarities. Create a function to calculate and compare the cosine of the angles for the three documents in each set. 
- [Step 4](#Step_4) Group reviewers for comparison, and create a for loop to calculate and print cosine similarities of reviewers.
- [Step 5](#Step_5) References. 

In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)
pd.set_option('max_columns', 120)
pd.set_option('max_colwidth', 5000)
pd.options.display.max_rows = 999
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings("ignore")

Back to [top](#top) 

<a id='Step_1'></a>

#### Step 1: Data wrangling.

Select text of reviewers that are being compared and combine all their reviews into a dataframe with different rows for each reviewer, and a column that is the entire text of their reviews joined into a single string. For each reviewer, create a variable representing the string of all their reviews joined together.  

In [2]:
file_loc = "C:/Users/rmbrm/Documents/DS_WP/data/critics.csv"
df = pd.read_csv(file_loc, index_col=None, na_values=['NA'], usecols = ['critic', 'quote'] )
df = df[~df.quote.isnull()]
df.head()

,critic,quote
1,Derek Adams,"So ingenious in concept, design and execution that you could watch it on a postage stamp-sized screen and still be engulfed by its charm."
2,Richard Corliss,The year's most inventive comedy.
3,David Ansen,A winning animated feature that has something for everyone on the age spectrum.
4,Leonard Klady,The film sports a provocative and appealing story that's every bit the equal of this technical achievement.
5,Jonathan Rosenbaum,"An entertaining computer-generated, hyperrealist animation feature (1995) that's also in effect a toy catalog."


In [3]:
reviewers = ["Anthony Lane", "Dave Kehr", "Rex Reed", "Jonathan Rosenbaum", "Richard Roeper", "Kenneth Turan",
             "Michael Wilmington", "David Edelstein", "Owen Gleiberman", "Lisa Schwarzbaum"]

df = df.loc[df['critic'].isin(reviewers)]

df2 = df.groupby('critic').agg({
    'quote': lambda x: ' '.join(x)
})

df2.head(2)

quote
critic                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [4]:
Lane= df2.loc["Anthony Lane", "quote"]
Kehr= df2.loc["Dave Kehr", "quote"]
Reed= df2.loc["Rex Reed", "quote"]
Rosenbaum= df2.loc["Jonathan Rosenbaum", "quote"]
Roeper= df2.loc["Richard Roeper", "quote"]
Turan= df2.loc["Kenneth Turan", "quote"]
Wilmington= df2.loc["Michael Wilmington", "quote"]
Edelstein= df2.loc["David Edelstein", "quote"]
Gleiberman= df2.loc["Owen Gleiberman", "quote"]
Schwarzbaum= df2.loc["Lisa Schwarzbaum", "quote"]



Back to [top](#top) 

<a id='Step_2'></a>

#### Step 2: Process the text for analysis.

Create a function to process text for analysis, including tokenizing (break words into components), stemming with nltk PorterStemmer (consolidate different permutations of words), and removing stopwords (specific articles, prepositions and other words that do not add meaning).


In [5]:
def process(file):
    raw = open(file).read()
    tokens = word_tokenize(raw)
    words = [w.lower() for w in tokens]
    
    porter = nltk.PorterStemmer()
    stemmed_tokens = [porter.stem(t) for t in words]
    
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [w for w in stemmed_tokens if not w in stop_words]
    
    count = nltk.defaultdict(int)
    for word in filtered_tokens:
        count[word] += 1
    return count;

Back to [top](#top) 

<a id='Step_3'></a>

#### Step 3: Compute cosine similarities.

Create a function to calculate and compare the cosine of the angles for the three documents in each set.

In [6]:
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

def getSimilarity(dict1, dict2):
    all_words_list = []
    for key in dict1:
        all_words_list.append(key)
    for key in dict2:
        all_words_list.append(key)
    all_words_list_size = len(all_words_list)
    
    v1 = np.zeros(all_words_list_size, dtype=np.int)
    v2 = np.zeros(all_words_list_size, dtype=np.int)
    i = 0
    for (key) in all_words_list:
        v1[i] = dict1.get(key, 0)
        v2[i] = dict2.get(key, 0)
        i = i + 1
    return cos_sim(v1, v2);

Back to [top](#top) 

<a id='Step_4'></a>

#### Step 4: Group reviewers for comparison, and create a for loop to calculate and print cosine similarities of reviewers.

In [7]:
sets = [["Lane", "Kehr", "Reed"], ["Lane", "Roeper", "Reed"], ["Kehr", "Rosenbaum", "Roeper"], ["Kehr", "Rosenbaum", "Turan"], ["Kehr", "Rosenbaum", "Wilmington"],
        ["Edelstein", "Gleiberman", "Wilmington"], ["Edelstein", "Gleiberman", "Roeper"], 
        ["Edelstein", "Gleiberman", "Lane"],["Edelstein", "Gleiberman", "Schwarzbaum"], 
       ["Edelstein", "Gleiberman","Reed"], ["Kehr", "Rosenbaum", "Edelstein"], 
        ["Lane", "Reed", "Wilmington"]]

In [8]:
numbers = range(0,12)
for num in numbers:
    rev1 = (sets[num])[0]
    rev2 = (sets[num])[1]
    rev3 = (sets[num])[2]
    dict1 = process(rev1)
    dict2 = process(rev2)
    dict3 = process(rev3)
    sim1 = (getSimilarity(dict1,dict2)).round(2)
    sim2 = (getSimilarity(dict1,dict3)).round(2)
    sim3 = (getSimilarity(dict2,dict3)).round(2)
    print(f"Similarity between {rev1} and {rev2} is {sim1}")
    print(f"Similarity between {rev1} and {rev3} is {sim2}")
    print(f"Similarity between {rev2} and {rev3} is {sim3}")
    print('\n')
    print('-----------------------')
    print('\n')
    

Similarity between Lane and Kehr is 0.92
Similarity between Lane and Reed is 0.89
Similarity between Kehr and Reed is 0.91


-----------------------


Similarity between Lane and Roeper is 0.79
Similarity between Lane and Reed is 0.89
Similarity between Roeper and Reed is 0.85


-----------------------


Similarity between Kehr and Rosenbaum is 0.97
Similarity between Kehr and Roeper is 0.88
Similarity between Rosenbaum and Roeper is 0.9


-----------------------


Similarity between Kehr and Rosenbaum is 0.97
Similarity between Kehr and Turan is 0.97
Similarity between Rosenbaum and Turan is 0.96


-----------------------


Similarity between Kehr and Rosenbaum is 0.97
Similarity between Kehr and Wilmington is 0.95
Similarity between Rosenbaum and Wilmington is 0.95


-----------------------


Similarity between Edelstein and Gleiberman is 0.94
Similarity between Edelstein and Wilmington is 0.93
Similarity between Gleiberman and Wilmington is 0.98


-----------------------


Similarit

Back to [top](#top) 

<a id='Step_5'></a>

#### Step 5: References

The primary references for this project are:

 - Schütze, Hinrich; Raghavan, Prabhakar; Manning, Christopher D. Introduction to Information Retrieval.
 - https://blogs.oracle.com/meena/finding-similarity-between-text-documents